In [1]:
import numpy as np # operaciones matemáticas (numerical python)
import pandas as pd # manipulación de datos
import warnings # nobody likes warnings


from surprise import accuracy

In [2]:
data5 = pd.read_csv('RAW_interactions.csv')

In [3]:
data6= pd.read_csv('RAW_recipes.csv')

U = user id
i = recipe id

In [4]:
data5.shape

(1132367, 5)

In [5]:
data5.columns


Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')

In [6]:
data5.head(5)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [7]:
data6.shape

(231637, 12)

In [8]:
data6.head(5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [9]:
ingredientes_unicos = data6['ingredients'].unique()
ingredientes_unicos

array(["['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",
       "['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']",
       "['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']",
       ...,
       "['hard-cooked eggs', 'mayonnaise', 'dijon mustard', 'salt-free cajun seasoning', 'tabasco sauce', 'salt', 'black pepper', 'fresh italian parsley']",
       "['butter', 'eagle brand condensed milk', 'light brown sugar', 'sour cream', 'egg', 'extract', 'nutmeg', 'self-rising flour', 'bisquick', 'wooden popsicle sticks']",
       "['granulated sugar', 'shortening', 'eggs', 'flour', 'cream of tartar', 'baking soda', 'vanilla extract']"],
      dtype=object)

In [10]:
nutrientes_unicos = data6['nutrition'].unique()
nutrientes_unicos

array(['[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]',
       '[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]',
       '[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]', ...,
       '[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]',
       '[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]',
       '[174.9, 14.0, 33.0, 4.0, 4.0, 11.0, 6.0]'], dtype=object)

In [11]:
ingredientes_unicos = data6['ingredients'].str.strip("[]").str.replace("'", "").str.split(", ", expand=True).stack().unique()

# Crear un nuevo DataFrame con los ingredientes únicos
tabla_ingredientes = pd.DataFrame({'ingredientes': ingredientes_unicos})

In [12]:
tabla_ingredientes.sample(25)

,ingredientes
11115,green pepper rings
8840,garlic bagel chips
177,margarine
11610,honey-nougat chocolate bar
296,beef steaks
10899,pan drippings from turkey
2849,cheez-it crackers
1282,strawberry jam
14105,canned salt-reduced spaghetti
14510,galanga powder


In [13]:
tabla_ingredientes.shape

(14906, 1)

In [14]:
data6['nutrition'] = data6['nutrition'].apply(eval)

# Crear nuevas columnas para cada número y valor
data6[['calories', 'total_fat_pdv', 'sugar_pdv', 'sodium_pdv', 'protein_pdv', 'saturated_fat', 'carbohydrates']] = pd.DataFrame(data6['nutrition'].tolist(), index=data6.index)


In [15]:
columnas_a_eliminar = ['contributor_id', 'submitted', 'tags','nutrition',]
data6 = data6.drop(columnas_a_eliminar, axis=1)


In [16]:
data6.sample(1)

,name,id,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat,carbohydrates
10413,aunt m fruit salad,151327,5,6,"['mix pudding with milk', 'fold in cool whip',...",this is the best fruit salad i've ever seen an...,"['instant vanilla pudding', 'milk', 'cool whip...",4,124.0,10.0,56.0,6.0,2.0,27.0,5.0


In [17]:
fila_deseada = data5[data5['recipe_id'] == 59389]

fila_deseada

,user_id,recipe_id,date,rating,review
287675,296809,59389,2008-06-11,4,"This is an excellent, easy-fix side-dish w/ton..."
287676,1232243,59389,2009-04-15,5,This is wonderful and super easy! I work with...


In [18]:
mean_ratings = data5.groupby('recipe_id')['rating'].mean().reset_index()

# Fusionar data6 y mean_ratings en data8 utilizando la columna 'recipe_id'
data8 = pd.merge(data6, mean_ratings, left_on='id', right_on='recipe_id', how='left')

# Renombrar la columna 'rating' para reflejar que ahora es la media de las notas
data8.rename(columns={'rating_x': 'original_rating', 'rating_y': 'mean_rating'}, inplace=True)

In [19]:
data8.columns

Index(['name', 'id', 'minutes', 'n_steps', 'steps', 'description',
       'ingredients', 'n_ingredients', 'calories', 'total_fat_pdv',
       'sugar_pdv', 'sodium_pdv', 'protein_pdv', 'saturated_fat',
       'carbohydrates', 'recipe_id', 'rating'],
      dtype='object')

In [20]:
data8.sample(1)

,name,id,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat,carbohydrates,recipe_id,rating
19721,beatty s chocolate cake barefoot contessa,362837,35,22,"['chocolate buttercream , recipe follows', 'pr...","don't skip the coffee, it's what makes this ca...","['butter', 'all-purpose flour', 'sugar', 'coco...",17,860.8,79.0,281.0,26.0,21.0,124.0,34.0,362837,4.333333


In [21]:
correction_dict = {
    '[butter': 'butter',
    'eggs': 'egg',
    # Añade más correcciones según sea necesario
}

# Función para corregir las palabras en una lista de ingredientes
def corregir_palabras(ingredientes):
    return [correction_dict.get(ingrediente, ingrediente) for ingrediente in ingredientes]

# Aplicar la función de corrección a la columna 'ingredients'
data8['ingredients'] = data8['ingredients'].apply(lambda x: corregir_palabras(x.split(', ')))

In [22]:

# Usar la función explode para convertir las listas en filas individuales
ingredients_df = data8[['recipe_id', 'ingredients']].explode('ingredients')

# Crear un DataFrame con los ingredientes únicos y su frecuencia
unique_ingredients_df = ingredients_df.groupby('ingredients').agg({'recipe_id': list}).reset_index()
unique_ingredients_df['num_recipes_containing_ingredient'] = unique_ingredients_df['recipe_id'].apply(len)



In [23]:
unique_ingredients_df.head(1)

,ingredients,recipe_id,num_recipes_containing_ingredient
0,"""adolph's meat tenderizer""","[487455, 149141, 382924, 12837]",4


In [24]:
unique_ingredients_df.columns

Index(['ingredients', 'recipe_id', 'num_recipes_containing_ingredient'], dtype='object')

In [25]:
data8.sample(1)

,name,id,minutes,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat,carbohydrates,recipe_id,rating
145610,nutella swirl pound cake,393144,85,20,"['1', 'preheat the oven to 325f lightly grease...",this came out in food & wine this month.\r\ni ...,"[['flour', 'eggs', 'vanilla extract', 'baking ...",8,448.7,39.0,125.0,6.0,11.0,76.0,16.0,393144,4.0


In [26]:
for column in data8.columns:
    column_type = data8[column].apply(type).unique()
    print(f"Columna '{column}' data type: {column_type}")

Columna 'name' data type: [<class 'str'> <class 'float'>]
Columna 'id' data type: [<class 'int'>]
Columna 'minutes' data type: [<class 'int'>]
Columna 'n_steps' data type: [<class 'int'>]
Columna 'steps' data type: [<class 'str'>]
Columna 'description' data type: [<class 'str'> <class 'float'>]
Columna 'ingredients' data type: [<class 'list'>]
Columna 'n_ingredients' data type: [<class 'int'>]
Columna 'calories' data type: [<class 'float'>]
Columna 'total_fat_pdv' data type: [<class 'float'>]
Columna 'sugar_pdv' data type: [<class 'float'>]
Columna 'sodium_pdv' data type: [<class 'float'>]
Columna 'protein_pdv' data type: [<class 'float'>]
Columna 'saturated_fat' data type: [<class 'float'>]
Columna 'carbohydrates' data type: [<class 'float'>]
Columna 'recipe_id' data type: [<class 'int'>]
Columna 'rating' data type: [<class 'float'>]


In [27]:
data8 = data8.dropna(subset=['name'])


In [28]:
data8.to_csv('recipes_data.csv', index=False)

In [29]:
data8['description'] = data8['description'].fillna('There is no description')

In [30]:
def explore_data(data):
    # Inicializa listas para almacenar resultados por columna
    duplicates_list = []
    nan_values_list = []
    empty_spaces_list = []

    # Itera sobre las columnas del DataFrame
    for column in data.columns:
        # Encuentra y cuenta duplicados basados en la representación de cadenas (excepto listas)
        duplicates = data[column].astype(str).duplicated().sum()
        duplicates_list.append(duplicates)

        # Encuentra y cuenta valores NaN (teniendo en cuenta listas)
        nan_values = data[column].apply(lambda x: any(pd.isna(y) for y in x) if isinstance(x, list) else pd.isna(x)).sum()
        nan_values_list.append(nan_values)

        # Encuentra y cuenta espacios en blanco (teniendo en cuenta listas)
        empty_spaces = data[column].apply(lambda x: any(y == ' ' for y in x) if isinstance(x, list) else x == ' ').sum()
        empty_spaces_list.append(empty_spaces)

    # Nuevo DataFrame con los resultados
    exploration = pd.DataFrame({"Duplicates": duplicates_list, "NaNValues": nan_values_list, "EmptySpaces": empty_spaces_list}, index=data.columns)

    # Muestra los resultados
    print(f"There are {sum(duplicates_list)} duplicates. Also;")
    
    return exploration

# Ejemplo de uso con tu DataFrame data8
explore_data_result = explore_data(data8)
print(explore_data_result)

There are 2528662 duplicates. Also;
               Duplicates  NaNValues  EmptySpaces
name                 1451          0            0
id                      0          0            0
minutes            230748          0            0
n_steps            231542          0            0
steps                 563          0            0
description          8968          0            0
ingredients          1162          0            0
n_ingredients      231595          0            0
calories           210158          0            0
total_fat_pdv      230716          0            0
sugar_pdv          229187          0            0
sodium_pdv         230746          0            0
protein_pdv        231004          0            0
saturated_fat      230439          0            0
carbohydrates      231123          0            0
recipe_id               0          0            0
rating             229260          0            0


In [31]:
unique_ingredients_df.columns

Index(['ingredients', 'recipe_id', 'num_recipes_containing_ingredient'], dtype='object')

In [32]:
data8.columns


Index(['name', 'id', 'minutes', 'n_steps', 'steps', 'description',
       'ingredients', 'n_ingredients', 'calories', 'total_fat_pdv',
       'sugar_pdv', 'sodium_pdv', 'protein_pdv', 'saturated_fat',
       'carbohydrates', 'recipe_id', 'rating'],
      dtype='object')

In [ ]:
# Interfaz de línea de comandos para que el usuario ingrese los ingredientes disponibles
ingredientes_disponibles = input("Ingrese los ingredientes disponibles separados por coma: ").split(',')
ingredientes_disponibles = [ingrediente.strip() for ingrediente in ingredientes_disponibles]

# Filtrar el DataFrame para encontrar recetas que contienen al menos uno de los ingredientes proporcionados
recetas = set()
for ingrediente in ingredientes_disponibles:
    # Convertir la entrada del usuario en minúsculas y eliminar comillas y otros caracteres no deseados
    ingrediente_limpio = ingrediente.lower().replace("'", "").strip()
    
    # Filtrar el DataFrame para encontrar recetas que contienen el ingrediente
    recetas_ingrediente = unique_ingredients_df[unique_ingredients_df['ingredients'].str.lower().str.contains(ingrediente_limpio)]['recipe_id']
    
    # Convertir cada lista de recipe_id en un conjunto antes de agregarlo al conjunto de recetas
    for ids in recetas_ingrediente:
        recetas.update(set(ids))

# Filtrar las recetas encontradas por los ingredientes proporcionados
recetas_encontradas = data8[data8['recipe_id'].isin(recetas)]

# Presentar los resultados al usuario
if recetas_encontradas.empty:
    print("Lo siento, no se encontraron recetas con los ingredientes proporcionados.")
else:
    # Seleccionar las columnas deseadas del DataFrame recetas_encontradas
    recetas_para_mostrar = recetas_encontradas.loc[:, ['name','recipe_id', 'rating', 'minutes', 'steps', 'description','calories','total_fat_pdv', 'sugar_pdv', 'sodium_pdv', 'protein_pdv', 'saturated_fat', 'carbohydrates']]
    
    # Ordenar las recetas según su rating de mayor a menor y luego mostrar las primeras 10
    recetas_ordenadas = recetas_para_mostrar.sort_values(by='rating', ascending=False)
recetas_ordenadas.head(1)

In [ ]:
merged_df = pd.merge(data5, data8, on='recipe_id')
merged_df.sample(5)

In [ ]:
merged_df.shape

In [ ]:
merged_df.columns

In [ ]:
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split  # Agregar esta línea
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

merged_df['gusto'] = 0
recetas_gustadas = []
while len(recetas_gustadas) < 5:
    receta_aleatoria = random.choice(merged_df['recipe_id'])
    receta_info = merged_df.loc[merged_df['recipe_id'] == receta_aleatoria, ['name', 'description']]
    print(f"Receta: {receta_info['name'].values[0]}")
    print(f"Descripción: {receta_info['description'].values[0]}")
    respuesta = input("¿Te gusta esta receta? (si/no): ").lower()
    if respuesta == "si":
        recetas_gustadas.append(receta_aleatoria)
        merged_df.loc[merged_df['recipe_id'] == receta_aleatoria, 'gusto'] = 1
        print("¡Genial!")
    elif respuesta == "no":
        merged_df.loc[merged_df['recipe_id'] == receta_aleatoria, 'gusto'] = 0
        print("Entendido, buscando otra receta...")
    else:
        print("Respuesta no válida. Por favor, responde 'si' o 'no'.")


In [ ]:
# Preguntar al usuario qué receta quiere hacer
receta_elegida = input("¿Qué receta te gustaría hacer? Por favor, escribe el nombre de la receta: ")

# Buscar la receta en el DataFrame
receta_elegida_info = merged_df.loc[merged_df['name'].str.lower() == receta_elegida.lower()]

# Verificar si la receta existe
if len(receta_elegida_info) == 0:
    print("Lo siento, no se encontró la receta. Por favor, asegúrate de escribir el nombre correctamente.")
else:
    # Mostrar la información de la receta
    print(f"Receta: {receta_elegida_info['name'].values[0]}")
    
    # Mostrar los pasos para hacer la receta en una línea horizontal
    print("Pasos para hacer la receta:")
    pasos_receta = ' '.join(receta_elegida_info['steps'].values[0])  # Unir los pasos en una sola cadena
    print(pasos_receta)
    
    # Mostrar información adicional
    print("\nInformación adicional de la receta:")
    print(f"Tiempo de preparación: {receta_elegida_info['minutes'].values[0]} minutos")
    print("Ingredientes:")
    for ingredient in receta_elegida_info['ingredients'].values[0]:
        print("- " + ingredient)
    print(f"Calorías: {receta_elegida_info['calories'].values[0]}")
    print(f"Grasa total (% del valor diario): {receta_elegida_info['total_fat_pdv'].values[0]}%")
    print(f"Azúcar (% del valor diario): {receta_elegida_info['sugar_pdv'].values[0]}%")
    print(f"Sodio (% del valor diario): {receta_elegida_info['sodium_pdv'].values[0]}%")
    print(f"Proteína (% del valor diario): {receta_elegida_info['protein_pdv'].values[0]}%")
    print(f"Grasa saturada: {receta_elegida_info['saturated_fat'].values[0]}")
    print(f"Carbohidratos: {receta_elegida_info['carbohydrates'].values[0]}")


In [ ]:
import pandas as pd

# Preguntar al usuario cuántas calorías necesita consumir al día y cuántas comidas realiza al día
calorias_diarias = int(input("¿Cuántas calorías necesita consumir al día? "))
comidas_al_dia = int(input("¿Cuántas comidas realiza al día? "))

# Calcular el número de calorías por comida
calorias_por_comida = calorias_diarias / comidas_al_dia

# Crear un DataFrame vacío para almacenar las recetas seleccionadas
recetas_seleccionadas = pd.DataFrame(columns=merged_df.columns)

# Inicializar la suma de calorías de las recetas seleccionadas
calorias_totales = 0

# Iterar hasta alcanzar o superar el número total de calorías necesarias
while calorias_totales < calorias_diarias:
    # Filtrar las recetas que tengan un contenido calórico por debajo o igual a las calorías por comida
    recetas_filtradas = merged_df[merged_df['calories'] <= calorias_por_comida]

    # Seleccionar una receta aleatoria de las que cumplen el criterio
    receta_seleccionada = recetas_filtradas.sample(n=1)

    # Agregar la receta seleccionada al DataFrame de recetas seleccionadas
    recetas_seleccionadas = pd.concat([recetas_seleccionadas, receta_seleccionada])

    # Actualizar la suma de calorías totales
    calorias_totales += receta_seleccionada['calories'].iloc[0]

# Mostrar el DataFrame con las recetas seleccionadas
print("Recetas seleccionadas para cumplir con el criterio de calorías por comida:")
recetas_seleccionadas

